In [1]:
import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pyfftw as ft
import time
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

In [2]:
N = 1024

In [3]:
Q = np.zeros((N,N,N),dtype='float32')
R = np.zeros((N,N,N),dtype='float32')
S2 = np.zeros((N,N,N), dtype='float32')

In [4]:
start = time.time()

nproc = 8
nx = N//nproc

lQ = np.zeros((N//nproc,N,N), dtype='float32')
lR = np.zeros((N//nproc,N,N), dtype='float32')

folder = '/home/jhelsas/scratch'

for k in range(nproc):
    filename = 'ref-Q-'+str(k)+'-(t=0.0).npz'
    files = folder+'/'+filename
    
    content = np.load(files)
    lQ = content['Q']
    
    Q[k*nx:(k+1)*nx,:,:] = lQ[:,:,:]
    
    filename = 'ref-R-'+str(k)+'-(t=0.0).npz'
    files = folder+'/'+filename
    
    content = np.load(files)
    lR = content['R']
    
    R[k*nx:(k+1)*nx,:,:] = lR[:,:,:]
    
end = time.time()
print(end-start)

15.8050229549


In [6]:
start = time.time()

nproc = 8
nx = N//nproc

lS2 = np.zeros((N//nproc,N,N), dtype='float32')

folder = '/home/jhelsas/scratch'

for k in range(nproc):
    filename = 'ref-strainrate-'+str(k)+'.npz'
    files = folder+'/'+filename
    
    content = np.load(files)
    lS2 = content['S2']
    
    S2[k*nx:(k+1)*nx,:,:] = lS2[:,:,:]
    
end = time.time()
print(end-start)

7.70472311974


In [7]:
del lQ,lR,lS2

In [8]:
avg = np.average(S2)
print(avg)

249.219


In [9]:
print(avg/2)

124.609298706


In [9]:
def FindJointPDF(x,y,lowerLim_x,upperLim_x,lowerLim_y,upperLim_y,bins_x,bins_y):
    x=x.flatten()
    y=y.flatten()
    totL=len(x)
    xedges=np.linspace(lowerLim_x,upperLim_x,bins_x+1,endpoint=True); dx=xedges[1]-xedges[0]
    yedges=np.linspace(lowerLim_y,upperLim_y,bins_y+1,endpoint=True); dy=yedges[1]-yedges[0]
    jPDF,xedges,yedges=np.histogram2d(x,y,bins=(xedges,yedges))
    jPDF=jPDF/(dx*dy*totL)
    gridX,gridY=np.meshgrid(xedges[0:-1]+0.5*dx,yedges[0:-1]+0.5*dy)
    return jPDF.T,gridX,gridY

In [12]:
t1=time.time()

rng=np.random.randint(N**3,size=(N**3)//8)

t2=time.time()
print('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 1.15 seconds



In [14]:
t1=time.time()
q=Q.flatten()[rng]
r=R.flatten()[rng]
q=q/avg
r=r/(avg**1.5)
t2=time.time()
print('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 7.96 seconds



In [ ]:
t1=time.time()
Ngrid=400;rmin=-25.0;rmax=25.0;qmin=-25.0;qmax=25.0 # Constants for jPDF
jPDF,gridX,gridY=FindJointPDF(r.flatten(),q.flatten(),rmin,rmax,qmin,qmax,Ngrid,Ngrid)
t2=time.time()
sys.stdout.write('Done!! Cost= %0.2f seconds\n' %(t2-t1))